In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Aca abajo está el código para descargar en formato .json una lista de métricas de WDI Bank Data

In [4]:
import requests
import json

metrics = ['NY.GDP.PCAP.PP.CD', 'SL.EMP.WORK.ZS']  # Add more metrics as needed

def fetch_data(metric):
    # URL of the API
    url = f"https://api.worldbank.org/v2/country/all/indicator/{metric}?format=jsonstat"

    # Make the HTTP request to the API and get the response in JSON format
    response = requests.get(url)
    json_data = response.json()

    # Assuming you already have the JSON response stored in a variable named json_data
    data = json_data["WDI"]["value"]
    seriesLabel = json_data["WDI"]["dimension"]["series"]["category"]["label"][metric]
    source = json_data["WDI"]["source"]
    updated = json_data["WDI"]["updated"]
    countryLabels = json_data["WDI"]["dimension"]["country"]["category"]["label"]
    years = list(json_data["WDI"]["dimension"]["year"]["category"]["label"].values())

    structuredData = []

    for i in range(len(data)):
        countryIndex = i // len(years)
        yearIndex = i % len(years)
        country = list(countryLabels.values())[countryIndex]
        year = years[yearIndex]
        value = data[i]

        dataPoint = {
            "Source": source,
            "Updated": updated,
            "Series": seriesLabel,
            "Country": country,
            "Year": year,
            "Value": value,
        }

        structuredData.append(dataPoint)

    # Now, structuredData contains the desired data structure with multiple countries
    return {"structuredData": structuredData}

# Loop through the list of metrics and save data in separate JSON files
for metric in metrics:
    data = fetch_data(metric)
    json_file_path = f"{metric}.json"  # Replace with your desired file path
    with open(json_file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)
    print(f"Data for {metric} saved to {json_file_path}")


Data for NY.GDP.PCAP.PP.CD saved to NY.GDP.PCAP.PP.CD.json
Data for SL.EMP.WORK.ZS saved to SL.EMP.WORK.ZS.json


Aca abajo está el código para descargar en formato .csv una tabla con dos métricas de WDI Bank data

In [8]:
import requests
import json
import pandas as pd

metrics = ['NY.GDP.PCAP.PP.CD', 'SL.EMP.WORK.ZS']  # Add more metrics as needed

def fetch_data(metric):
    # URL of the API
    url = f"https://api.worldbank.org/v2/country/all/indicator/{metric}?format=jsonstat"

    # Make the HTTP request to the API and get the response in JSON format
    response = requests.get(url)
    json_data = response.json()

    # Assuming you already have the JSON response stored in a variable named json_data
    data = json_data["WDI"]["value"]
    seriesLabel = json_data["WDI"]["dimension"]["series"]["category"]["label"][metric]
    source = json_data["WDI"]["source"]
    updated = json_data["WDI"]["updated"]
    countryLabels = json_data["WDI"]["dimension"]["country"]["category"]["label"]
    years = list(json_data["WDI"]["dimension"]["year"]["category"]["label"].values())

    structuredData = []

    for i in range(len(data)):
        countryIndex = i // len(years)
        yearIndex = i % len(years)
        country = list(countryLabels.values())[countryIndex]
        year = years[yearIndex]
        value = data[i]

        dataPoint = {
            "Source": source,
            "Updated": updated,
            "Series": seriesLabel,
            "Country": country,
            "Year": year,
            f"Value_{metric}": value,
        }

        structuredData.append(dataPoint)

    # Now, structuredData contains the desired data structure for one metric
    return structuredData

# Initialize an empty list to store data for all metrics
combined_data = []

# Loop through the list of metrics and fetch data
for metric in metrics:
    metric_data = fetch_data(metric)
    combined_data.extend(metric_data)

# Create a DataFrame from the combined data
df_combined = pd.DataFrame(combined_data)

# Save the combined data to a CSV file or do further processing as needed
df_combined.to_csv('financeEvents1_chart1.csv', index=False)
